<a href="https://colab.research.google.com/github/AbbasFAlhokri/AbbasFAlhokri/blob/main/prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
# Importing the required data 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import date
import seaborn as sn
import matplotlib.pyplot as plt
# You can configure the format of the images: ‘png’, ‘retina’, ‘jpeg’, ‘svg’, ‘pdf’.
%config InlineBackend.figure_format = 'svg'
# this statement allows the visuals to render within your Jupyter Notebook
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
import pycaret 
from pycaret.classification import *

In [3]:
# Load data that we prepared 
df = pd.read_csv('preprocessed_data.csv')
df.head()

,hour,day_of_week,month,year,day_off,temp,rain_1h,snow_1h,clouds_all,Clear,Clouds,Drizzle,Fog,Haze,Mist,Rain,Smoke,Snow,Squall,Thunderstorm,traffic_volume
0,9,1,10,2012,0,15.13,0.0,0.0,40.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5545.0
1,10,1,10,2012,0,16.21,0.0,0.0,75.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4516.0
2,11,1,10,2012,0,16.43,0.0,0.0,90.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4767.0
3,12,1,10,2012,0,16.98,0.0,0.0,90.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5026.0
4,13,1,10,2012,0,17.99,0.0,0.0,75.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4918.0


From the previous analysis we found the following corrolation metrix with traffic_volunm.
traffic_volume    1.000000
hour              0.355087
temp              0.139241
clouds_all        0.078210
Clear             0.052370
Clear             0.052370
Clear             0.052370
year              0.010384
Rain              0.008778
Rain              0.008778
Rain              0.008778
Thunderstorm      0.004931
Thunderstorm      0.004931
Thunderstorm      0.004931
Drizzle           0.001099
Drizzle           0.001099
Drizzle           0.001099
Smoke             0.000990
Smoke             0.000990
Smoke             0.000990
snow_1h          -0.002091
month            -0.004865
Squall           -0.005900
Squall           -0.005900
Squall           -0.005900
Haze             -0.010891
Haze             -0.010891
Haze             -0.010891
rain_1h          -0.013425
Clouds           -0.033277
Clouds           -0.033277
Clouds           -0.033277
Fog              -0.036782
Fog              -0.036782
Fog              -0.036782
Mist             -0.037965
Mist             -0.037965
Mist             -0.037965
Snow             -0.038194
Snow             -0.038194
Snow             -0.038194
holiday          -0.044235
day_of_week      -0.144165
day_off          -0.215541

In [7]:
# we notice from the above corrolation metrix, hour , temp, cloud_all, day_off and day_of_week are the most corrolated to target value.
X = df[['day_off', 'temp', 'clouds_all', 'hour','day_of_week']]
y = df['traffic_volume']

In [59]:
# Create train and test sets for the chosen feature.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Create train and test sets for all features!
X_all_train, X_all_test, y_all_train, y_all_test = train_test_split(df, y, test_size=0.2, random_state=42)

In [68]:
# since we are training on different models let's creat a function that takes model's name, and required paramters. 
def train_test_various_models(models, X_train, X_test, y_train, y_test):
    """ we will pass models as a dictionary to train 
    train and test given modele on given data"""
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        print(f"r2 score on test data using {model_name}: {r2_score(y_test, y_test_pred)}\n")
        print(f"mean squared error score on test data using {model_name}:", end=" "),
        print(f"{np.sqrt(mean_squared_error(y_test, y_test_pred))}\n")

In [56]:
# train and test various models.
models = {
    'linear_regression': LinearRegression(),
    'decision_tree_regresor': DecisionTreeRegressor(max_depth=5 , random_state = 0),
    'random_forest_regressor': RandomForestRegressor(max_depth=5, n_estimators=10, random_state = 0)
}

train_test_various_models(models, X_train, X_test, y_train, y_test)

In [58]:
# Let's see the result if we train the model with all features!
train_test_various_models(models, X_all_train, X_all_test, y_all_train, y_all_test)

r2 score on test data using linear_regression: 1.0

mean squared error score on test data using linear_regression: 3.9630581729659173e-13

r2 score on test data using decision_tree_regresor: 0.999017556121947

mean squared error score on test data using decision_tree_regresor: 62.253273605591964

r2 score on test data using random_forest_regressor: 0.9993656683909454

mean squared error score on test data using random_forest_regressor: 50.02262694285912



# standardizing train and test data
.


In [64]:
# Create pipelines for data preparing
class DataFrameSelector(BaseEstimator, TransformerMixin):
    """Class from book Handson-ml by Ayrelien Geron. This estimators
    takes chosen columns from dataframe"""
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [69]:
# Define piplines and use them on a x_train and x_test to standarize numerical variables
#  'clouds_all', 'hour','day_of_week'
numerical_attributs = ['hour', 'temp']
categorical_attributs = ['clouds_all', 'day_off']

numerical_pipeline = Pipeline([
    ('selector', DataFrameSelector(numerical_attributs)),
    ('std_scaler', StandardScaler())
])
categorical_pipeline = Pipeline([
    ('selector', DataFrameSelector(categorical_attributs)),
])
full_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline', numerical_pipeline),
    ('cat_pipeline', categorical_pipeline)
])

X_train_std = full_pipeline.fit_transform(X_train)
X_test_std = full_pipeline.transform(X_test)

KeyError: ignored

In [39]:
from pycaret.regression import *
#intialize the setup
exp_reg = setup(df, target = 'traffic_volume')

,Description,Value
0,session_id,5475
1,Target,traffic_volume
2,Original Data,"(40555, 21)"
3,Missing Values,False
4,Numeric Features,5
5,Categorical Features,15
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(28388, 43)"


In [40]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,247.7581,1.828171e+05,427.2453,0.9535,0.2654,0.7185,0.310
rf,Random Forest Regressor,256.0733,2.142243e+05,462.4442,0.9455,0.2557,0.7545,12.032
et,Extra Trees Regressor,283.8734,2.768064e+05,525.8288,0.9296,0.2736,0.6839,12.293
gbr,Gradient Boosting Regressor,367.5803,3.112882e+05,557.3500,0.9209,0.3435,0.8576,2.638
dt,Decision Tree Regressor,338.2017,3.896166e+05,623.6674,0.9010,0.3477,0.6697,0.212
knn,K Neighbors Regressor,477.7126,5.121761e+05,715.5800,0.8697,0.3679,0.9180,0.367
ada,AdaBoost Regressor,619.1005,7.039654e+05,838.5519,0.8209,0.3803,0.9156,0.878
lr,Linear Regression,1540.7090,3.112835e+06,1764.2688,0.2081,0.8893,2.3559,0.522
ridge,Ridge Regression,1540.7559,3.112791e+06,1764.2565,0.2081,0.8896,2.3559,0.044
br,Bayesian Ridge,1542.2320,3.113378e+06,1764.4218,0.2079,0.8892,2.3589,0.124


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=5475, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)